# Anemômetro utilizando termistor

Este notebook modela um anemômetro utilizando um termistor aquecido pela corrente elétrica passando por ele e esfriado pelo escoamento ao redor do mesmo.

A resistência elétrica do termistor varia de acordo com a seguinte relação

$$
R = R_0\exp\left[ B \cdot \left(\frac{1}{T} - \frac{1}{T_0} \right) \right]
$$

Onde 
 * $T$ é a temperatura do termistor em K
 * $T_0$ é a temperatura de referência em K
 * $R_0$ é a resistência do termistor à temperature de referência $T_0$
 * $B$ é um coeficiente empírico com unidade 1/K
 

In [ ]:
using Plots
pyplot()

In [ ]:
include("../src/ThermistorHW.jl")
using ThermistorHW

## Verificando o comportamento do Termistor NTC

Vamos modelar um termistor com as seguintes características:

 * $T_0 = 20^\circ C$
 * $R_0 = 5000\:\Omega$
 * $B = 3200\:K^{-1}$
 

In [ ]:
R = Thermistor(5e3, 3000, 20)

In [ ]:
T = 0.0:0.5:120.0
r = R.(T);

In [ ]:
plot(T, r, xlab="Temperature (°C)", ylab="Resistance (Ω)",
    label="Thermistor R₀ = 5kΩ, B = 3000 K", title="Resistance of a Thermistor")

savefig("../figures/thermistor.pdf")



## Funcionamento de um anomômetro

O calor gerado pela corrente elétrica no termistor é transferido por convecção para o escoamento ao seu redor:

$$
R(T)\cdot I^2 = h\cdot A\cdot \left(T - T_\infty\right)
$$

Lembrando que $h$ depende da velocidade, esta equação pode ser resolvida. Existem dois modos básicos de operação:

 * Corrente constante, onde I é conhecida e deve-se determinar a temperatura do termistor
 * Temperatura constante, onde T é conhecida e deve-se determinar a corrente
 
 É interessante observar que apesar da eletrônica do modo corrente constante ser muito mais simples, neste modelo é necessário usar um esquema iterativo para calcular a temperatura. Por outro lado, o modelo com temperatura constante é direto.

## Temperatura constante

In [ ]:
R = Thermistor(5e3, 3000)
Pa = 93.0 # kPa, pressão atmosférica
Ta = 20.0 # Temperatura ambiente
U = 0.5:0.1:30


In [ ]:
d1 = 1.5
d2 = 2.0
d3 = 3.0
Tw1 = 70
Tw2 = 80
Tw3 = 90

a11 = const_temperature.(Tw1, R, U, d1, Ta, Pa)
a12 = const_temperature.(Tw1, R, U, d2, Ta, Pa)
a13 = const_temperature.(Tw1, R, U, d3, Ta, Pa)

a21 = const_temperature.(Tw2, R, U, d1, Ta, Pa)
a22 = const_temperature.(Tw2, R, U, d2, Ta, Pa)
a23 = const_temperature.(Tw2, R, U, d3, Ta, Pa)

a31 = const_temperature.(Tw3, R, U, d1, Ta, Pa)
a32 = const_temperature.(Tw3, R, U, d2, Ta, Pa)
a33 = const_temperature.(Tw3, R, U, d3, Ta, Pa)

b1 = const_temperature.(Tw1, R, U, d2, Ta+1, Pa)
b2 = const_temperature.(Tw2, R, U, d2, Ta+1, Pa)
b3 = const_temperature.(Tw3, R, U, d2, Ta+1, Pa)

E11 = [x.E for x in a11]
I11 = [1000*x.I for x in a11]
E12 = [x.E for x in a12]
I12 = [1000*x.I for x in a12]
E13 = [x.E for x in a13]
I13 = [1000*x.I for x in a13]

E21 = [x.E for x in a21]
I21 = [1000*x.I for x in a21]
E22 = [x.E for x in a22]
I22 = [1000*x.I for x in a22]
E23 = [x.E for x in a23]
I23 = [1000*x.I for x in a23]

E31 = [x.E for x in a31]
I31 = [1000*x.I for x in a31]
E32 = [x.E for x in a32]
I32 = [1000*x.I for x in a32]
E33 = [x.E for x in a33]
I33 = [1000*x.I for x in a33];

E1b = [x.E for x in b1]
I1b = [1000*x.I for x in b1]
E2b = [x.E for x in b2]
I2b = [1000*x.I for x in b3]
E3b = [x.E for x in b3]
I3b = [1000*x.I for x in b3];



In [ ]:
plot(U, E21, xlab="Velocity (m/s)", 
    ylab="Voltage (V)", title="", 
    label="T = 80°C, D = 1.5 mm", color=:red, linestyle=:solid)
plot!(U, E22, label="T = 80°C, D = 2 mm", color=:green, linestyle=:solid)
plot!(U, E23, label="T = 80°C, D = 3 mm", color=:black, linestyle=:solid)

plot!(U, E31, label="T = 90°C, D = 1.5 mm", color=:red, linestyle=:dash)
plot!(U, E32, label="T = 90°C, D = 2 mm", color=:green, linestyle=:dash)
plot!(U, E33, label="T = 90°C, D = 3 mm", color=:black, linestyle=:dash)


#savefig("../figures/CTA-Eo.pdf")

In [ ]:
plot(U, E21, xlab="Velocity (m/s)", 
    ylab="Voltage (V)", title="", 
    label="T = 80°C, D = 1.5 mm", color=:red, linestyle=:solid)
plot!(U, E22, label="T = 80°C, D = 2 mm", color=:green, linestyle=:solid)
plot!(U, E23, label="T = 80°C, D = 3 mm", color=:black, linestyle=:solid)

plot!(U, E31, label="T = 90°C, D = 1.5 mm", color=:red, linestyle=:dash)
plot!(U, E32, label="T = 90°C, D = 2 mm", color=:green, linestyle=:dash)
plot!(U, E33, label="T = 90°C, D = 3 mm", color=:black, linestyle=:dash)


savefig("../figures/CTA-Eo.pdf")

In [ ]:
plotattr("ls")

In [ ]:
plot(U, I21, xlab="Velocity (m/s)", 
    ylab="Current (mA)", title="", 
    label="T = 80°C, D = 1.5 mm", color=:red, linestyle=:solid)
plot!(U, I22, label="T = 80°C, D = 2 mm", color=:green, linestyle=:solid)
plot!(U, I23, label="T = 80°C, D = 3 mm", color=:black, linestyle=:solid)

plot!(U, I31, label="T = 90°C, D = 1.5 mm", color=:red, linestyle=:dash)
plot!(U, I32, label="T = 90°C, D = 2 mm", color=:green, linestyle=:dash)
plot!(U, I33, label="T = 90°C, D = 3 mm", color=:black, linestyle=:dash)

#savefig("../figures/CTA-I.pdf")

In [ ]:
plot(U, I21, xlab="Velocity (m/s)", 
    ylab="Current (mA)", title="", 
    label="T = 80°C, D = 1.5 mm", color=:red, linestyle=:solid)
plot!(U, I22, label="T = 80°C, D = 2 mm", color=:green, linestyle=:solid)
plot!(U, I23, label="T = 80°C, D = 3 mm", color=:black, linestyle=:solid)

plot!(U, I31, label="T = 90°C, D = 1.5 mm", color=:red, linestyle=:dash)
plot!(U, I32, label="T = 90°C, D = 2 mm", color=:green, linestyle=:dash)
plot!(U, I33, label="T = 90°C, D = 3 mm", color=:black, linestyle=:dash)

savefig("../figures/CTA-I.pdf")

In [ ]:
e1 = -(E1b - E12) ./ E12 * 100
e2 = -(E2b - E22) ./ E22 * 100
e3 = -(E3b - E32) ./ E32 * 100




plot(U, e1, xlab="Velocity (m/s)", 
    ylab="Voltage difference (%)", label="T = 70°C")

plot!(U, e2, label="T = 80°C")
plot!(U, e3, label="T = 90°C")

In [ ]:
plot(U, e1, xlab="Velocity (m/s)", 
    ylab="Voltage difference (%)", label="T = 70°C")

plot!(U, e2, label="T = 80°C")
plot!(U, e3, label="T = 90°C")
savefig("../figures/CTA-dt.pdf")

In [ ]:
p1 = plot(U, E1, xlab="Velocity (m/s)", 
        ylab="Voltage (V)", title="(a) Voltage output", 
    label="T = 90°C, D = 2 mm")
plot!(U, E2, label="T=90°C, D = 3 mm")
plot!(U, E3, label="T = 80°C, D = 2 mm")

p2 = plot(U, ea, xlab="Velocity (m/s)", 
    ylab="Voltage Difference (%)", title="(b) Effect of room temp.",
    label="T = 90°C")
plot!(U, eb, label="T = 80°C")

pp = plot(p1,p2, layout=(1,2), legend=false)


savefig(pp, "../figures/CTA.pdf")

### Conclusões

Os gráficos acima mostram que o comportamento do sensor é não linear e 

$$
E \sim \sqrt{U}
$$
Isto é interessante pois quanto menor a velocidade, mais sensível é o instrumento. Por outro lado, a convecção natural limita isso. 

Os gráficos acima mostram que quanto maior a temperatura de operação maior a sensibilidade mas maior a corrente. Por outro lado, quanto menor a temperatura, maior a sensibilidade em relação à temperatura ambiente o que é ruim

Por outro lado reduzir o diâmetro possui uma vantagem considerável: limitam-se as tensões e principalmente a corrente. Isto é muito interessante para sistemas que trabalham com baterias.



## Corrente constante

In [ ]:
R = Thermistor(5e3, 3000)
Pa = 93.0 # kPa, pressão atmosférica
Ta = 20.0 # Temperatura ambiente
U = 0.5:0.1:30


In [ ]:
d1 = 1.5
d2 = 2.0
d3 = 3.0

i1 = 10.0
i2 = 15.0
i3 = 20.0


c11 = const_current.(i1, R, U, d1, Ta, Pa); 
c12 = const_current.(i1, R, U, d2, Ta, Pa); 
c13 = const_current.(i1, R, U, d3, Ta, Pa); 

c21 = const_current.(i2, R, U, d1, Ta, Pa); 
c22 = const_current.(i2, R, U, d2, Ta, Pa); 
c23 = const_current.(i2, R, U, d3, Ta, Pa); 

c31 = const_current.(i3, R, U, d1, Ta, Pa); 
c32 = const_current.(i3, R, U, d2, Ta, Pa); 
c33 = const_current.(i3, R, U, d3, Ta, Pa); 

cb1 = const_current.(i1, R, U, d2, Ta+1.0, Pa); 
cb2 = const_current.(i2, R, U, d2, Ta+1.0, Pa); 
cb3 = const_current.(i3, R, U, d2, Ta+1.0, Pa); 



Ei11 = [x.E for x in c11];
Ei12 = [x.E for x in c12];
Ei13 = [x.E for x in c13];

Ei21 = [x.E for x in c21];
Ei22 = [x.E for x in c22];
Ei23 = [x.E for x in c23];

Ei31 = [x.E for x in c31];
Ei32 = [x.E for x in c32];
Ei33 = [x.E for x in c33];

Ei1b = [x.E for x in cb1];
Ei2b = [x.E for x in cb2];
Ei3b = [x.E for x in cb3];


T11 = [x.T for x in c11];
T12 = [x.T for x in c12];
T13 = [x.T for x in c13];

T21 = [x.T for x in c21];
T22 = [x.T for x in c22];
T23 = [x.T for x in c23];

T31 = [x.T for x in c31];
T32 = [x.T for x in c32];
T33 = [x.T for x in c33];

Tb1 = [x.T for x in cb1];
Tb2 = [x.T for x in cb2];
Tb3 = [x.T for x in cb3];



ei1 = (Ei12 - Ei1b) ./ Ei12 * 100;
ei2 = (Ei22 - Ei2b) ./ Ei22 * 100;
ei3 = (Ei32 - Ei3b) ./ Ei32 * 100;



In [ ]:
plot(U, Ei12, xlab="Velocity (m/s)", 
    ylab="Voltage (V)",
    label="I = 10 mA, D = 2 mm", color=:red)
plot!(U, Ei22, label="I = 15 mA, D = 2 mm", color=:green)
plot!(U, Ei32, label="I = 20 mA, D = 2 mm", color=:black)

#plot!(U, Ei13, label="I = 10 mA, D = 3 mm", color=:red, linestyle=:dash)
#plot!(U, Ei23, label="I = 15 mA, D = 3 mm", color=:green, linestyle=:dash)
#plot!(U, Ei33, label="I = 20 mA, D = 3 mm", color=:black, linestyle=:dash)



In [ ]:
c12[end].T

In [ ]:
plot(U, Ei12, xlab="Velocity (m/s)", 
    ylab="Voltage (V)",
    label="I = 10 mA, D = 2 mm", color=:red)
plot!(U, Ei22, label="I = 15 mA, D = 2 mm", color=:green)
plot!(U, Ei32, label="I = 20 mA, D = 2 mm", color=:black)

savefig("../figures/CCA-Eo.pdf")

In [ ]:
plot(U, T12, xlab="Velocity (m/s)", 
    ylab="Temperature (°C)",label="I = 10 mA", color=:red)
plot!(U, T22, label="I = 15 mA", color=:green)
plot!(U, T32, label="I = 20 mA", color=:black)

In [ ]:
plot(U, T12, xlab="Velocity (m/s)", 
    ylab="Temperature (°C)",label="I = 10 mA", color=:red)
plot!(U, T22, label="I = 15 mA", color=:green)
plot!(U, T32, label="I = 20 mA", color=:black)

savefig("../figures/CCA-T.pdf")

In [ ]:
plot(U, ea, xlab="Velocidade (m/s)", 
    ylab="Diferença de tensão (%)", title="Efeito de variação de 2oC da temperatura ambiente",
    label="I = 25 mA")
plot!(U, eb, label="I = 20 mA")
plot!(U, ec, label="I = 15 mA")

In [ ]:
p1 = plot(U, E1, xlab="Velocity (m/s)", 
    ylab="Voltage (V)", title="Voltage output", 
    label="I = 25 mA")
plot!(U, E2, label="I = 20 mA")
plot!(U, E5, label="I = 15 mA")



p2 = plot(U, T1, xlab="Velocity (m/s)", 
    ylab="Temperature (°C)", title="Thermistor temperature", 
    label="I = 25 mA")
plot!(U, T2, label="I = 20 mA")
plot!(U, T5, label="I = 15 mA")

pp = plot(p1, p2, layout=(1,2))
savefig(pp, "../figures/CCA.pdf")


### Conclusões

O comportamento é bem parecido em geral com o caso com temperatura constante. Talvez os aspecto mais importante seja a variação acentuada da temperatura o que pode causar algumas dificuldades. Um problema é a temperatura excessiva em baixas velocidades o que pode causar danos ao termistor mas atingir altas temperaturas é o único jeito de se conseguir sensibilidade nas velocidades mais altas.